# electricity_market_player

> Fill in a module description here

In [ ]:
#| default_exp electricity_market_player

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import matplotlib.pyplot as plt
import numpy as np
import optuna
import seaborn as sns

from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from scipy import stats
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from rliable import metrics, plot_utils, library as rly

from electricity_market.electricity_market_env import ElectricityMarketEnv


In [ ]:
def collect_episode_rewards(model, env, n_episodes=10, deterministic=True):
    episode_rewards, _ = evaluate_policy(
        model, env, deterministic=deterministic, use_masking=True,
        return_episode_rewards=True, n_eval_episodes=n_episodes
    )
    return episode_rewards


def mask_fn(env):
    return env.action_masks()


seeds = [123456, 234567, 345678, 456789, 567890]
env_config = {}
results = {}

In [ ]:
def evaluate_maskable_ppo_on_environment(hyperparameters=None):
    global seeds

    if hyperparameters is None:
        hyperparameters = {}

    frames = np.array(list(range(1000, 10001, 1000)), dtype=int)

    all_rewards = []

    for seed in seeds:
        print(f"\nRunning experiment with seed {seed}...")
        env = DummyVecEnv([
            lambda: Monitor(ActionMasker(ElectricityMarketEnv(env_config), mask_fn))
        ])

        model = MaskablePPO(
            MaskableActorCriticPolicy,
            env,
            verbose=0,
            seed=seed,
            **hyperparameters
        )

        seed_rewards = []

        for frame in frames:
            model.learn(total_timesteps=frame, use_masking=True, reset_num_timesteps=False)
            rewards = collect_episode_rewards(model, env, n_episodes=10, deterministic=True)
            seed_rewards.append(rewards)

        seed_rewards = np.array(seed_rewards)  # Shape: (num_checkpoints, num_episodes)
        all_rewards.append(seed_rewards)

    all_rewards = np.array(all_rewards)  # Shape: (num_seeds, num_checkpoints, num_episodes)
    print("\nCollected Rewards (shape: seeds x checkpoints x episodes):\n", all_rewards)
    return all_rewards

In [ ]:
def plot_evaluation_results(evaluation_results):
    # Extract algorithm names (which are actually keys in the dictionary)
    algorithms = list(evaluation_results.keys())

    # Function to compute aggregate metrics (median, IQM, mean) for each checkpoint and seed
    def aggregate_func(x):
        return np.array([
            metrics.aggregate_median(x),
            metrics.aggregate_iqm(x),
            metrics.aggregate_mean(x),
        ])

    # For each algorithm, we need to apply aggregate_func to the data (which has the shape (num_seeds, num_checkpoints, num_episodes))
    def aggregate_over_checkpoints(evaluation_results):
        aggregated_results = {}
        for algorithm, results in evaluation_results.items():
            # results.shape is (num_seeds, num_checkpoints, num_episodes)
            # We aggregate across seeds and episodes for each checkpoint
            agg_results = np.array([aggregate_func(results[:, i, :]) for i in range(results.shape[1])])
            aggregated_results[algorithm] = agg_results
        return aggregated_results

    # Aggregate results across seeds and episodes
    aggregated_results = aggregate_over_checkpoints(evaluation_results)

    # Use rly to compute interval estimates
    aggregate_scores, aggregate_score_cis = rly.get_interval_estimates(
        aggregated_results, aggregate_func, reps=50000
    )

    # Plot aggregate metrics (Median, IQM, Mean)
    metric_names = ['Median', 'IQM', 'Mean']
    fig, axes = plot_utils.plot_interval_estimates(
        aggregate_scores,
        aggregate_score_cis,
        metric_names=metric_names,
        algorithms=algorithms,
        xlabel='Reward'
    )
    plt.suptitle("Aggregate Metrics with 95% Stratified Bootstrap CIs")
    plt.show()

    # =============================================================================
    # 2. Probability of Improvement (if comparing two algorithms)
    # =============================================================================
    if len(algorithms) == 2:
        alg1, alg2 = algorithms
        algorithm_pairs = {f"{alg1},{alg2}": (evaluation_results[alg1], evaluation_results[alg2])}

        average_probabilities, average_prob_cis = rly.get_interval_estimates(
            algorithm_pairs, metrics.probability_of_improvement, reps=2000
        )

        plot_utils.plot_probability_of_improvement(average_probabilities, average_prob_cis)
        plt.title(f"Probability of Improvement: {alg1} vs {alg2}")
        plt.show()

    # =============================================================================
    # 3. Sample Efficiency Curve (using frames as defined in the evaluation function)
    # =============================================================================
    frames = np.array(list(range(1000, 10001, 1000)), dtype=int)
    sample_efficiency_dict = {
        alg: results[:, 1:, :]  # We want to remove the first checkpoint as it's usually 0
        for alg, results in evaluation_results.items() if len(results.shape) == 3
    }

    # Define the IQM function
    iqm_func = lambda scores: np.array([metrics.aggregate_iqm(scores[:, :, frame]) for frame in range(scores.shape[2])])

    # Compute IQM scores and confidence intervals using rly
    iqm_scores, iqm_cis = rly.get_interval_estimates(sample_efficiency_dict, iqm_func, reps=50000)
    print(f"Frames: {frames}")
    print(f"IQM Scores: {iqm_scores}")
    print(f"IQM CIs: {iqm_cis}")
    # Plot the sample efficiency curve
    plot_utils.plot_sample_efficiency_curve(
        frames=frames + 1,  # Adjust frames if necessary
        point_estimates=iqm_scores,
        interval_estimates=iqm_cis,
        algorithms=sample_efficiency_dict.keys(),
        xlabel='Number of Frames (in thousands)',
        ylabel='IQM Reward'
    )
    plt.title("Sample Efficiency Curve")
    plt.show()


    # =============================================================================
    # 4. Performance Profiles (linear and non-linear scaling)
    # =============================================================================
    thresholds = np.linspace(0.0, 8.0, 81)  # Define the thresholds
    score_distributions, score_distributions_cis = rly.create_performance_profile(
        evaluation_results, thresholds
    )

    # Plot performance profiles with linear scale
    fig, ax = plt.subplots(ncols=1, figsize=(7, 5))
    plot_utils.plot_performance_profiles(
        score_distributions,
        thresholds,
        performance_profile_cis=score_distributions_cis,
        colors=dict(zip(algorithms, sns.color_palette('colorblind'))),
        xlabel=r'Normalized Score $(\tau)$',
        ax=ax
    )
    plt.title("Performance Profiles (Linear Scale)")
    plt.show()

    # Plot performance profiles with non-linear scaling
    fig, ax = plt.subplots(ncols=1, figsize=(7, 5))
    plot_utils.plot_performance_profiles(
        score_distributions,
        thresholds,
        performance_profile_cis=score_distributions_cis,
        use_non_linear_scaling=True,
        xticks=[0.0, 0.5, 1.0, 2.0, 4.0, 8.0],
        colors=dict(zip(algorithms, sns.color_palette('colorblind'))),
        xlabel=r'Normalized Score $(\tau)$',
        ax=ax
    )
    plt.title("Performance Profiles (Non-Linear Scaling)")
    plt.show()


In [ ]:
def optimize_maskable_ppo_agent(trial):
    global seeds
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    n_steps = trial.suggest_int('n_steps', 16, 2048, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 256, log=True)

    trial_seed_rewards = []

    for seed in seeds:
        env = DummyVecEnv([
            lambda: Monitor(ActionMasker(ElectricityMarketEnv(env_config), mask_fn))
        ])

        model = MaskablePPO(
            MaskableActorCriticPolicy,
            env,
            learning_rate=learning_rate,
            n_steps=n_steps,
            batch_size=batch_size,
            verbose=0,
            seed=seed
        )

        model.learn(total_timesteps=int(1e5), use_masking=True)
        episode_rewards = collect_episode_rewards(model, env, n_episodes=10, deterministic=True)

        seed_avg_reward = np.mean(episode_rewards)
        trial_seed_rewards.append(seed_avg_reward)
    aggregated_performance = stats.trim_mean(trial_seed_rewards, proportiontocut=0.25)

    return aggregated_performance

### Evaluation MaskablePPO with default hyperparameters on ElectricityMarketEnv

In [ ]:
results["MaskablePPO_Baseline"] = evaluate_maskable_ppo_on_environment(hyperparameters=None)


Running experiment with seed 123456...


### Hypertuning MaskablePPO with default hyperparameters on ElectricityMarketEnv

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(optimize_maskable_ppo_agent, n_trials=10)

print("Best trial:", study.best_trial)

### Evaluation MaskablePPO with optimized hyperparameters on ElectricityMarketEnv

In [ ]:
results["MaskablePPO_Optimized"] = evaluate_maskable_ppo_on_environment(hyperparameters=study.best_trial.params)

In [ ]:
plot_evaluation_results(results)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()